# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import joblib

## Reload the smart sample here

In [2]:
# Reload your smart sampling from local file 
# ----------------------------------

sampled_X, sampled_y, model = joblib.load('sample-data-v1.pkl')

## Normalize/standardize the data if required

In [3]:
from sklearn.preprocessing import MinMaxScaler

processor = MinMaxScaler()
scaled_X = processor.fit_transform(sampled_X)

## Split the data into Train/Test

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_X, sampled_y, test_size=0.20)

## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a model

We are free to use any of the models that we learned in the past or use new models. 

* It is difficult to fit an anomaly detection method in the sklearn pipeline without writing custom codes. For simplicity, we avoid fitting an anomaly detection method within a pipeline. So we can create the workflow in two steps. 
    * Step I: fit an outlier with the training set
    * Step II: define a pipeline using a feature selection and a classification method. Then cross-validate this pipeline using the training data without outliers. 
        * Note: if your smart sample is somewhat imbalanced, you might want to change the scoring method in GridSearchCV (see the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

* Once we fit the pipeline, we identify the best model and give an unbiased evaluation using the test set that we created in Part II. For unbiased evaluation we report confusion matrix, precision, recall, f1-score, accuracy, and other measures if you like. 

(Optional) Those who are interested in writing custom codes for adding an outlier detection method into the sklearn pipeline, please follow this discussion [thread](https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline). 


**Note:** <span style='background:yellow'>We will be using Grid Search to find the optimal parameters of the pipelines.</span>

You can add more notebook cells or import any Python modules as needed.

In [5]:
from sklearn.svm import OneClassSVM, SVC
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest, RandomForestClassifier

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif, RFECV

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


### Your 1st pipeline 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation
  
Add cells as needed. 

In [6]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------
env = EllipticEnvelope().fit(X_train, y_train)
env_outliers = env.predict(X_train)==-1
X_clean = X_train[~env_outliers]
y_clean = y_train[~env_outliers]


In [7]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------

pipe1 = Pipeline([
    ('SKB', SelectKBest(chi2)),
    ('Logistic Regression', LogisticRegression(max_iter=5000, solver='saga'))
])

param_grid = {'SKB__k':[5,8,10,12,16],
             'Logistic Regression__C':[0.1,10,100,200]
             }

model_grid1 = GridSearchCV(pipe1, param_grid, cv=10)
model_grid1.fit(X_clean,y_clean)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('SKB',
                                        SelectKBest(score_func=<function chi2 at 0x7f9c387b0048>)),
                                       ('Logistic Regression',
                                        LogisticRegression(max_iter=5000,
                                                           solver='saga'))]),
             param_grid={'Logistic Regression__C': [0.1, 10, 100, 200],
                         'SKB__k': [5, 8, 10, 12, 16]})

In [8]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
pred_y = model_grid1.predict(X_test)

model_grid1.score(X_test, y_test)
print(classification_report(y_test, pred_y))
pd.DataFrame(confusion_matrix(y_test, pred_y))


              precision    recall  f1-score   support

           0       0.61      0.63      0.62      2299
           1       0.60      0.58      0.59      2219

    accuracy                           0.60      4518
   macro avg       0.60      0.60      0.60      4518
weighted avg       0.60      0.60      0.60      4518



,0,1
0,1442,857
1,938,1281


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

In [9]:
model_grid1.best_estimator_

Pipeline(steps=[('SKB',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f9c387b0048>)),
                ('Logistic Regression',
                 LogisticRegression(C=200, max_iter=5000, solver='saga'))])

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [10]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------
iso = IsolationForest().fit(X_train, y_train)
iso_outliers = iso.predict(X_train)==-1
X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]

In [20]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------

pipe2 = Pipeline([
    ('SKB', SelectKBest()),
    ('DTC', DecisionTreeClassifier())
])

param_grid = {'SKB__k':[1,5,8,10,12,16]}

model_grid2 = GridSearchCV(pipe2, param_grid, cv=10)
model_grid2.fit(X_iso,y_iso)


/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [20] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [20] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: UserWarning: Features [20] are constant.
  UserWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/opt/conda/lib/python3.7/site-packages/skle

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('SKB', SelectKBest()),
                                       ('DTC', DecisionTreeClassifier())]),
             param_grid={'SKB__k': [1, 5, 8, 10, 12, 16]})

In [21]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------

pred_y = model_grid2.predict(X_test)

model_grid2.score(X_test, y_test)
print(classification_report(y_test, pred_y))
pd.DataFrame(confusion_matrix(y_test, pred_y))


              precision    recall  f1-score   support

           0       0.87      0.86      0.86      2299
           1       0.85      0.87      0.86      2219

    accuracy                           0.86      4518
   macro avg       0.86      0.86      0.86      4518
weighted avg       0.86      0.86      0.86      4518



,0,1
0,1967,332
1,299,1920


In [22]:
model_grid2.best_estimator_

Pipeline(steps=[('SKB', SelectKBest(k=16)), ('DTC', DecisionTreeClassifier())])

#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [14]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------
svm = OneClassSVM(kernel='rbf').fit(X_train, y_train)
svm_outliers = svm.predict(X_train)==-1
X_svm = X_train[~svm_outliers]
y_svm = y_train[~svm_outliers]

In [15]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------

pipe3 = Pipeline([
    ('PCA', PCA(svd_solver='full')),
    ('RFC', RandomForestClassifier()),
])

param_grid = {'PCA__n_components':[5,10,15,20],
             'RFC__n_estimators':[10,50,100]}
model_grid3 = GridSearchCV(pipe3, param_grid, cv=10)
model_grid3.fit(X_svm,y_svm)


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('PCA', PCA(svd_solver='full')),
                                       ('RFC', RandomForestClassifier())]),
             param_grid={'PCA__n_components': [5, 10, 15, 20],
                         'RFC__n_estimators': [10, 50, 100]})

In [16]:
print(model_grid3.best_estimator_)

Pipeline(steps=[('PCA', PCA(n_components=20, svd_solver='full')),
                ('RFC', RandomForestClassifier())])


In [17]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------

pred_y = model_grid3.predict(X_test)

model_grid3.score(X_test, y_test)
print(classification_report(y_test, pred_y))
pd.DataFrame(confusion_matrix(y_test, pred_y))


              precision    recall  f1-score   support

           0       0.83      0.75      0.79      2299
           1       0.77      0.84      0.80      2219

    accuracy                           0.80      4518
   macro avg       0.80      0.80      0.79      4518
weighted avg       0.80      0.80      0.79      4518



,0,1
0,1726,573
1,353,1866


#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## Compare these three pipelines and discuss your findings

## <span style="background: yellow;">Commit your code!</span> 

### Pickle the required pipeline/models for Part III.

In [23]:

import joblib
joblib.dump([pipe2, model_grid2], 'data-part-3.pkl')



['data-part-3.pkl']

You should have made a few commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Pipelines done`


# Save your notebook!
## Then `File > Close and Halt`